In [1]:
# prompt: 구글드라이브 연동해줘

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 라이브러리 선언

In [2]:
# 원하는 경로로 이동
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/data_analysis_2025/MLProject')

# 현재 경로 확인
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/data_analysis_2025/MLProject


In [42]:
## api 호출 라이브러
import requests
## 데이터 조작 라이브러리
import pandas as pd
import numpy as np
## 머신러닝 라이브러리
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings(action="ignore")
import xgboost as xgb
##전처리 라이브러리
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
## 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
!pip install -U scikit-learn
!pip install -U imbalanced-learn
!pip install -U xgboost

## 데이터 불러오기

In [5]:
df = pd.read_csv('Churn_Modelling.csv')

In [6]:
df.columns = df.columns.str.upper()
df

,ROWNUMBER,CUSTOMERID,SURNAME,CREDITSCORE,GEOGRAPHY,GENDER,AGE,TENURE,BALANCE,NUMOFPRODUCTS,HASCRCARD,ISACTIVEMEMBER,ESTIMATEDSALARY,EXITED
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9998,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


## 타입통합/특성 숫자컬럼 추가

In [7]:
# HasCrCard, IsActiveMember, Age, Geography는 결측치가 존재

In [8]:
# 결측치 제거(소량이라 없애도 될거 같음)
df = df.dropna()

In [9]:
oldCol = ["ROWNUMBER", "CUSTOMERID", "SURNAME"]
df = df.drop(oldCol, axis = 1)

In [10]:
strColumnList = ["GEOGRAPHY", "GENDER"]
intColumnList = ["CREDITSCORE", "AGE", "TENURE", "NUMOFPRODUCTS","HASCRCARD", "ISACTIVEMEMBER", "EXITED"]
floatColumnList = ["BALANCE", "ESTIMATEDSALARY"]

In [11]:
for i in range(0,len(strColumnList)):
    targetColumn = strColumnList[i]
    #df[targetColumn] = df[targetColumn].fillna('N')
    df[targetColumn] = df[targetColumn].astype(str)
for i in range(0,len(intColumnList)):
    targetColumn = intColumnList[i]
    #df[targetColumn] = df[targetColumn].fillna('0')
    df[targetColumn] = df[targetColumn].astype(int)
for i in range(0,len(floatColumnList)):
    targetColumn = floatColumnList[i]
    #df[targetColumn] = df[targetColumn].fillna('0')
    df[targetColumn] = df[targetColumn].astype(float)

In [12]:
# 결측치 제거 잘 됐는지 확인
print(df.isnull().sum())

CREDITSCORE        0
GEOGRAPHY          0
GENDER             0
AGE                0
TENURE             0
BALANCE            0
NUMOFPRODUCTS      0
HASCRCARD          0
ISACTIVEMEMBER     0
ESTIMATEDSALARY    0
EXITED             0
dtype: int64


In [13]:
def get_age_group(age):
    if age < 30:
        return 'young'
    elif age < 50:
        return 'middle'
    else:
        return 'senior'
    return None # 예외 처리

In [14]:
df["AGEGROUP"] = df["AGE"].apply(get_age_group)

In [15]:
genderMap = {"Male":0,"Female":1}
df["GENDER_LE"] = df.GENDER.map( genderMap )

In [16]:
# 잔고는 높지만 비활동 고객
df["HBLA"] = (
    (df["BALANCE"] > df["BALANCE"].median()) &
    (df["ISACTIVEMEMBER"] == 0)
).astype(int)

# 상품수가 낮은 고객
df["SPUSER"] = (df["NUMOFPRODUCTS"] == 1).astype(int)

# 신용등급 구분
df["CREDITGRADE"] = pd.qcut(df["CREDITSCORE"], q=4, labels=[4,3,2,1]).astype(int)

In [17]:
df

,CREDITSCORE,GEOGRAPHY,GENDER,AGE,TENURE,BALANCE,NUMOFPRODUCTS,HASCRCARD,ISACTIVEMEMBER,ESTIMATEDSALARY,EXITED,AGEGROUP,GENDER_LE,HBLA,SPUSER,CREDITGRADE
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,middle,1,0,1,3
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,middle,1,0,1,3
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,middle,1,1,0,4
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,middle,1,0,0,2
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1,middle,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1,middle,1,0,1,2
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,middle,0,0,0,1
9999,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,middle,0,0,0,1
10000,792,France,Female,28,4,130142.79,1,1,0,38190.78,0,young,1,1,1,1


In [18]:
# groupkeys = ["AGEGROUP", "GENDER"]

# groupData = df.groupby(by=groupkeys).agg(
#     cntExited=('EXITED', 'count'),             # 각 그룹의 개수
#     avgBalance=('BALANCE', 'mean'),    # 각 그룹의 평균 잔고
#     avgSalary=('ESTIMATEDSALARY', 'mean'), # 각 그룹의 급여 중앙값
#     ChurnRate=('EXITED', 'mean')          # 각 그룹의 이탈률
# )

In [19]:
# joinkey = ["AGEGROUP", "GENDER"]
# mergedData = pd.merge(left=df, right=groupData, on=joinkey)
# mergedData.head(20)

In [20]:
# 범주형 변수 원-핫 인코딩 (새로운 조합된 피처 포함)
# categorical_features = ['GEOGRAPHY', 'GENDER', 'AGEGROUP', 'TENUREGROUP', 'GEO_GEN', 'AGE_ACTV', 'NUMP_HCRCRD']
# featuresData = pd.get_dummies(featuresData, columns=categorical_features, drop_first=True)

## minmaxscaler 만들기

In [21]:
numeric_df = ["CREDITSCORE","BALANCE","ESTIMATEDSALARY"]
scaler = MinMaxScaler()
df[numeric_df] = scaler.fit_transform(df[numeric_df])

In [22]:
# scrScaler = MinMaxScaler()
# balScaler = MinMaxScaler()
# esrScaler = MinMaxScaler()

# featuresData["CREDITSCORE_SC"] = balScaler.fit_transform(featuresData.loc[:, ["CREDITSCORE"]] )
# featuresData["BALANCE_SC"] = balScaler.fit_transform(featuresData.loc[:, ["BALANCE"]] )
# featuresData["ESTIMATEDSALARY_SC"] = balScaler.fit_transform(featuresData.loc[:, ["ESTIMATEDSALARY"]] )

In [23]:
featuresData = df.loc[:]

## 특성선정 및 데이터분리

In [24]:
corrDf = featuresData.corr(numeric_only=True)
stdCorr = 0.15
label = ["EXITED"]
features =list(corrDf.loc[(abs(corrDf.EXITED) >= stdCorr) & (corrDf.EXITED != 1)].index)
print(f"label : {label} 선정된 features : {features}")

label : ['EXITED'] 선정된 features : ['AGE', 'ISACTIVEMEMBER', 'HBLA', 'SPUSER']


In [25]:
featuresData.shape

(9998, 16)

In [26]:
trainData, testData = train_test_split(featuresData, test_size=0.2, random_state=10)

In [27]:
trainDataFeatures = trainData.loc[:, features]
trainDataLabels = trainData.loc[:, label]
trainDataAll = trainData.loc[:]
testDataFeatures = testData.loc[:, features]
testDataLabels = testData.loc[:, label]
testDataAll = testData.loc[:]

In [28]:
print(trainDataFeatures.shape)
print(trainDataLabels.shape)
print(testDataFeatures.shape)
print(testDataLabels.shape)
print(testDataAll.shape)

(7998, 4)
(7998, 1)
(2000, 4)
(2000, 1)
(2000, 16)


## 모델 적용

In [29]:
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings(action="ignore")

In [51]:
knnModel = KNeighborsClassifier(n_neighbors=3)
scvModel = SVC(C=1.0)
dtModel = DecisionTreeClassifier(random_state=10)
rfModel = RandomForestClassifier(n_estimators=10, random_state=42, n_jobs=-1)

In [52]:
fittedKnnModel = knnModel.fit(trainDataFeatures, trainDataLabels)
fittedDtModel = dtModel.fit( X=trainDataFeatures, y=trainDataLabels )
fittedScvModel = scvModel.fit(trainDataFeatures, trainDataLabels)
fittedRfModel = rfModel.fit( X=trainDataFeatures, y=trainDataLabels.values.ravel() )

## 예측

In [53]:
predictKnnValue = fittedKnnModel.predict(testDataFeatures)
predictSvcValue = fittedScvModel.predict(testDataFeatures)
predictDtValue = fittedDtModel.predict(testDataFeatures)
predictRfValue = fittedRfModel.predict(testDataFeatures)
testDataAll["predict_knn"] = predictKnnValue
testDataAll["predict_svc"] = predictSvcValue
testDataAll["predict_dt"] = predictDtValue
testDataAll["predict_rf"] = predictRfValue

In [54]:
testDataAll

,CREDITSCORE,GEOGRAPHY,GENDER,AGE,TENURE,BALANCE,NUMOFPRODUCTS,HASCRCARD,ISACTIVEMEMBER,ESTIMATEDSALARY,...,HBLA,SPUSER,CREDITGRADE,predict_knn,predict_svc,predict_dt,PREDICT_DT,PREDICT_RF,PREDICT_LR,predict_rf
941,0.692,France,Female,31,8,0.000000,2,0,0,0.955404,...,0,0,2,0,0,0,0.034483,0.038412,0.097408,0
9357,0.136,France,Female,46,9,0.000000,1,1,1,0.405053,...,0,1,4,0,0,0,0.354167,0.348206,0.275053,0
5341,0.592,France,Female,51,4,0.405062,1,0,0,0.652710,...,1,1,3,1,0,1,0.764706,0.772606,0.502515,1
196,0.532,Spain,Female,32,6,0.000000,2,1,1,0.214970,...,0,0,3,0,0,0,0.070000,0.072168,0.000078,0
8054,0.350,Germany,Female,47,1,0.470660,1,1,0,0.440588,...,1,1,4,1,0,1,0.607143,0.604879,0.458722,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5627,0.608,France,Male,29,4,0.529915,1,1,1,0.733588,...,0,1,2,0,0,0,0.086957,0.088103,0.088930,0
5881,0.888,Spain,Male,31,0,0.577447,2,0,1,0.878243,...,0,0,1,0,0,0,0.021053,0.018682,-0.010871,0
991,0.530,France,Female,35,2,0.388365,2,1,1,0.699089,...,0,0,3,0,0,0,0.043011,0.042778,0.032923,0
6155,0.818,France,Male,26,4,0.000000,2,1,0,0.676980,...,0,0,1,0,0,0,0.000000,0.000000,0.042666,0


## 데이터 정리

In [46]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [57]:
knnAccuracy = accuracy_score( y_true=testDataAll.EXITED, y_pred = testDataAll.predict_knn)
svcAccuracy = accuracy_score( y_true=testDataAll.EXITED, y_pred = testDataAll.predict_svc)
dtAccuracy = accuracy_score( y_true=testDataAll.EXITED, y_pred = testDataAll.predict_dt)
rfAccuracy = accuracy_score( y_true=testDataAll.EXITED, y_pred = testDataAll.predict_rf)

In [58]:
accDf = pd.DataFrame( [[knnAccuracy, svcAccuracy, dtAccuracy, rfAccuracy]] , columns=["knn_accuracy","svc_accuracy","dt_accuracy", "rf_accuracy"])

In [59]:
accDf

,knn_accuracy,svc_accuracy,dt_accuracy,rf_accuracy
0,0.8165,0.802,0.8325,0.8345


## 예측 결과 저장

In [61]:
import joblib
modelDump = {}
modelDump["model"] = fittedScvModel
modelDump["features"] = features
modelDump["label"] = label
modelDump["preprocessing_feature"] = [scaler]
# modelDump["preprocessing_label"] = ynDict
joblib.dump(modelDump,"mlcore.dump")

['mlcore.dump']